<a href="https://colab.research.google.com/github/drissdunn/UmojaHack-2022/blob/main/umjhintermediate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# UmojaHack Africa 2022: Monthly Insurance Claim Prediction Challenge (INTERMEDIATE)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import  OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
pd.options.display.float_format = '{:.5f}'.format

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [62]:
# Load files
train = pd.read_csv('/content/drive/MyDrive/Datasets/UMJH/Main intermediate/train.csv')
policies = pd.read_csv('/content/drive/MyDrive/Datasets/UMJH/Main intermediate/policies.csv')
test = pd.read_csv('/content/drive/MyDrive/Datasets/UMJH/Main intermediate/test.csv')
sampsub = pd.read_csv('/content/drive/MyDrive/Datasets/UMJH/Main intermediate/SampleSubmission.csv')

In [63]:
train.head()

,claim_id,policy_number,claim_number,sum_insured,product,agent,class_of_business,risk_type,loss_date,client_type,renewal_frequency,primary_cause,secondary_cause,branch,target
0,ID_JZ7QWRHH57RO,pol000009680,claim0011359,300000.00000,prod00027,ag00068,cob00031,rt00006,2021-05-13,ct0003,rf0001,pc0007,sc00022,br00006,82150.00000
1,ID_KQ5TJMVLJ4EP,pol000006869,claim0005272,16000.00000,prod00005,ag00226,cob00010,rt00032,2018-05-13,ct0003,rf0001,pc0007,sc00023,br00002,3780.24000
2,ID_NYHI7WJGGIAE,pol000006135,claim0004147,3000.00000,prod00029,ag00037,cob00010,rt00026,2018-08-18,ct0003,rf0001,pc0007,sc00024,br00002,2196.50000
3,ID_ROMMDCXYUXN5,pol000008991,claim0009962,6000.00000,prod00005,ag00037,cob00009,rt00032,2018-12-20,ct0003,rf0001,pc0007,sc00021,br00001,105.67000
4,ID_2OTD9NX8L73D,pol000007768,claim0007346,26850.00000,prod00015,ag00226,cob00034,rt00023,2018-11-07,ct0001,rf0001,pc0007,sc00021,br00001,1605.64000


In [64]:
policies.head()

,policy_number,product_code,trans_date,from_date,to_date,agent,class_of_business,client_type,renewal_frequency,branch
0,pol000006995,prod00033,2017-07-01 08:24:55.800,2017-07-01 00:00:00.000,2018-07-01 00:00:00.000,ag00005,cob00035,ct0001,rf0002,br00002
1,pol000009828,prod00005,2017-07-01 09:04:02.217,2017-07-03 00:00:00.000,2017-11-02 00:00:00.000,ag00226,cob00009,ct0003,rf0001,br00006
2,pol000009828,prod00005,2017-07-01 09:04:02.217,2017-07-03 00:00:00.000,2017-11-02 00:00:00.000,ag00226,cob00031,ct0003,rf0001,br00006
3,pol000007729,prod00005,2017-07-01 09:36:53.843,2017-07-01 00:00:00.000,2017-10-31 00:00:00.000,ag00005,cob00031,ct0003,rf0001,br00002
4,pol000007729,prod00005,2017-07-01 09:36:53.843,2017-07-01 00:00:00.000,2017-10-31 00:00:00.000,ag00005,cob00009,ct0003,rf0001,br00002


In [65]:
train.columns

Index(['claim_id', 'policy_number', 'claim_number', 'sum_insured', 'product',
       'agent', 'class_of_business', 'risk_type', 'loss_date', 'client_type',
       'renewal_frequency', 'primary_cause', 'secondary_cause', 'branch',
       'target'],
      dtype='object')

In [66]:
train.dtypes

claim_id              object
policy_number         object
claim_number          object
sum_insured          float64
product               object
agent                 object
class_of_business     object
risk_type             object
loss_date             object
client_type           object
renewal_frequency     object
primary_cause         object
secondary_cause       object
branch                object
target               float64
dtype: object

In [67]:
train.isnull().sum()

claim_id              0
policy_number        86
claim_number          0
sum_insured           0
product               0
agent                 0
class_of_business     0
risk_type             0
loss_date             0
client_type           0
renewal_frequency     0
primary_cause         0
secondary_cause       0
branch                0
target                0
dtype: int64

In [68]:
train['policy_number'].mode()

0    pol000006026
dtype: object

In [69]:
test['policy_number'].mode()

0    pol000006026
dtype: object

In [70]:
test['policy_number'] = test['policy_number'].fillna(value = 'pol000006026')

In [71]:
train['policy_number'] = train['policy_number'].fillna(value = 'pol000006026')

In [72]:
test.isnull().sum()

claim_id             0
policy_number        0
claim_number         0
sum_insured          0
product              0
agent                0
class_of_business    0
risk_type            0
loss_date            0
client_type          0
renewal_frequency    0
primary_cause        0
secondary_cause      0
branch               0
dtype: int64

In [73]:
train.shape, test.shape

((9376, 15), (4019, 14))

In [74]:
train.dtypes

claim_id              object
policy_number         object
claim_number          object
sum_insured          float64
product               object
agent                 object
class_of_business     object
risk_type             object
loss_date             object
client_type           object
renewal_frequency     object
primary_cause         object
secondary_cause       object
branch                object
target               float64
dtype: object

In [75]:
train['policy_number'].unique()

array(['pol000009680', 'pol000006869', 'pol000006135', ...,
       'pol000006715', 'pol000006847', 'pol000007450'], dtype=object)

In [76]:
train['policy_number'] = train['policy_number'].str.strip('pol')
train['policy_number'] = train['policy_number'].astype(int)

In [77]:
test['policy_number'] = test['policy_number'].str.strip('pol')
test['policy_number'] = test['policy_number'].astype(int)

In [78]:
train['claim_number'] = train['claim_number'].str.strip('claim')
train['claim_number'] = train['claim_number'].astype(int)

In [79]:
test['claim_number'] = test['claim_number'].str.strip('claim')
test['claim_number'] = test['claim_number'].astype(int)

In [80]:
train['product'] = train['product'].str.strip('prod')
train['product'] = train['product'].astype(int)

In [81]:
test['product'] = test['product'].str.strip('prod')
test['product'] = test['product'].astype(int)

In [82]:
test['agent'] = test['agent'].str.strip('ag')
test['agent'] = test['agent'].astype(int)

In [83]:
train['agent'] = train['agent'].str.strip('ag')
train['agent'] = train['agent'].astype(int)

In [84]:
train['class_of_business'] = train['class_of_business'].str.strip('cob')
train['class_of_business'] = train['class_of_business'].astype(int)

In [85]:
test['class_of_business'] = test['class_of_business'].str.strip('cob')
test['class_of_business'] = test['class_of_business'].astype(int)

In [86]:
train['risk_type'] = train['risk_type'].str.strip('rt')
train['risk_type'] = train['risk_type'].astype(int)

In [87]:
test['risk_type'] = test['risk_type'].str.strip('rt')
test['risk_type'] = test['risk_type'].astype(int)

In [88]:
train['client_type'] = train['client_type'].str.strip('ct')
train['client_type'] = train['client_type'].astype(int)

In [89]:
test['client_type'] = test['client_type'].str.strip('ct')
test['client_type'] = test['client_type'].astype(int)

In [90]:
train['renewal_frequency'] = train['renewal_frequency'].str.strip('rf')
train['renewal_frequency'] = train['renewal_frequency'].astype(int)

In [91]:
test['renewal_frequency'] = test['renewal_frequency'].str.strip('rf')
test['renewal_frequency'] = test['renewal_frequency'].astype(int)

In [92]:
test['primary_cause'] = test['primary_cause'].str.strip('pc')
test['primary_cause'] = test['primary_cause'].astype(int)

In [93]:
train['secondary_cause'] = train['secondary_cause'].str.strip('sc')
train['secondary_cause'] = train['secondary_cause'].astype(int)

In [94]:
test['secondary_cause'] = test['secondary_cause'].str.strip('sc')
test['secondary_cause'] = test['secondary_cause'].astype(int)

In [95]:
train['branch'] = train['branch'].str.strip('br')
train['branch'] = train['branch'].astype(int)

In [96]:
test['branch'] = test['branch'].str.strip('br')
test['branch'] = test['branch'].astype(int)

In [97]:
train['primary_cause'] = train['primary_cause'].str.strip('pc')
train['primary_cause'] = train['primary_cause'].astype(int)

In [98]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9376 entries, 0 to 9375
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   claim_id           9376 non-null   object 
 1   policy_number      9376 non-null   int64  
 2   claim_number       9376 non-null   int64  
 3   sum_insured        9376 non-null   float64
 4   product            9376 non-null   int64  
 5   agent              9376 non-null   int64  
 6   class_of_business  9376 non-null   int64  
 7   risk_type          9376 non-null   int64  
 8   loss_date          9376 non-null   object 
 9   client_type        9376 non-null   int64  
 10  renewal_frequency  9376 non-null   int64  
 11  primary_cause      9376 non-null   int64  
 12  secondary_cause    9376 non-null   int64  
 13  branch             9376 non-null   int64  
 14  target             9376 non-null   float64
dtypes: float64(2), int64(11), object(2)
memory usage: 1.1+ MB


In [99]:
train.loss_date = pd.to_datetime(train.loss_date, errors='coerce')
test.loss_date = pd.to_datetime(test.loss_date, errors='coerce')

In [100]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9376 entries, 0 to 9375
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   claim_id           9376 non-null   object        
 1   policy_number      9376 non-null   int64         
 2   claim_number       9376 non-null   int64         
 3   sum_insured        9376 non-null   float64       
 4   product            9376 non-null   int64         
 5   agent              9376 non-null   int64         
 6   class_of_business  9376 non-null   int64         
 7   risk_type          9376 non-null   int64         
 8   loss_date          9376 non-null   datetime64[ns]
 9   client_type        9376 non-null   int64         
 10  renewal_frequency  9376 non-null   int64         
 11  primary_cause      9376 non-null   int64         
 12  secondary_cause    9376 non-null   int64         
 13  branch             9376 non-null   int64         
 14  target  

In [101]:
# Extract day, month and year from the loss date column
# day
train['loss_date_day'] = train.loss_date.dt.day

# month
train['loss_date_month'] = train.loss_date.dt.month
 
# year
train['loss_date_year'] = train.loss_date.dt.year

# Preview engineered date features
train[['loss_date', 'loss_date_day', 'loss_date_month', 'loss_date_year']].head()

,loss_date,loss_date_day,loss_date_month,loss_date_year
0,2021-05-13,13,5,2021
1,2018-05-13,13,5,2018
2,2018-08-18,18,8,2018
3,2018-12-20,20,12,2018
4,2018-11-07,7,11,2018


In [102]:
# Extract day, month and year from the loss date column
# day
test['loss_date_day'] = test.loss_date.dt.day

# month
test['loss_date_month'] = test.loss_date.dt.month
 
# year
test['loss_date_year'] = test.loss_date.dt.year

# Preview engineered date features
test[['loss_date', 'loss_date_day', 'loss_date_month', 'loss_date_year']].head()

,loss_date,loss_date_day,loss_date_month,loss_date_year
0,2021-03-11,11,3,2021
1,2020-06-25,25,6,2020
2,2019-03-14,14,3,2019
3,2018-12-04,4,12,2018
4,2019-09-18,18,9,2019


In [103]:
train = train.drop(['loss_date', 'claim_id'], axis = 1)
test = test.drop(['loss_date', 'claim_id'], axis = 1)

In [104]:
x = train.drop('target', axis = 1).values
y = train.target.values

In [105]:
test = test.values

In [106]:
x_train, x_test, y_train, y_test = train_test_split(x, y)

In [107]:
lr = LinearRegression()
lr.fit(x_train, y_train)

LinearRegression()

In [108]:
rf = RandomForestRegressor()
rf.fit(x_train, y_train)

RandomForestRegressor()

In [109]:
!pip install catboost

In [110]:
from catboost import CatBoostRegressor
ct = CatBoostRegressor()
ct.fit(x_train, y_train)

Learning rate set to 0.055721
0:	learn: 208233.0942863	total: 4.8ms	remaining: 4.79s
1:	learn: 206742.9102000	total: 7.34ms	remaining: 3.66s
2:	learn: 205124.3105143	total: 10ms	remaining: 3.32s
3:	learn: 203682.8568173	total: 12.5ms	remaining: 3.12s
4:	learn: 202364.9031283	total: 15.3ms	remaining: 3.05s
5:	learn: 200661.6464150	total: 18.1ms	remaining: 2.99s
6:	learn: 199589.6243288	total: 21ms	remaining: 2.98s
7:	learn: 198509.4537174	total: 23.4ms	remaining: 2.9s
8:	learn: 197117.3474929	total: 25.9ms	remaining: 2.85s
9:	learn: 196020.7494968	total: 28.8ms	remaining: 2.85s
10:	learn: 195079.3622377	total: 31.8ms	remaining: 2.86s
11:	learn: 193935.1181126	total: 34.5ms	remaining: 2.84s
12:	learn: 193003.1503421	total: 36.9ms	remaining: 2.8s
13:	learn: 192000.8099827	total: 39.7ms	remaining: 2.79s
14:	learn: 191144.1386513	total: 42.4ms	remaining: 2.78s
15:	learn: 190245.3719840	total: 45ms	remaining: 2.77s
16:	learn: 189423.9329863	total: 47.7ms	remaining: 2.76s
17:	learn: 188783.12

In [111]:
def report(model):
    pred = model.predict(x_test)
    print(f'The mean square error is {mean_squared_error(pred, y_test)}')
    print(f'The r2_score is {r2_score(pred, y_test)}')

In [116]:
pred = ct.predict(test)

In [118]:
submission = sampsub.copy()
submission.target = pred
submission.to_csv('abc1.csv', index=False)